# Классификация последовательностей (текстов) -- good practices

На прошлом семинаре мы рассмотрели пример генерации текстов. Однако мы не обсудили весь огромный функционал, который предоставляет пайторч для работы с текстами. В частности работали мы с посимвольной генерацией, а не пословной/потокенной. Исправляемся!

Сегодня речь зайдёт о задаче классификации текстовых последовательностей. Для этого будем пользоваться датасетом IMDB и библиотекой `torchtext`. В торчтексте реализовано огромное число методов для обработки текстов, ими мы и воспользуемся.

__АХТУНГ__. Торчтекст не рекомендуется использовать для обучения на больших данных (от миллиона примеров и больше) из-за маленькой скорости работы. В таких случаях рекомендуется имплементировать свои датасеты. В наших примерах таких объёмов данных не будет.

In [ ]:
# import transformers

## Начнём с модели.

Да, не самая привыичная последовательность действий, но давайте сначала сделаем кое-что, с чем мы уже знакомы -- имплементируем модель.

На вход она будет принимать батч последовательностей токенов, разумеется разной длины в разных батчах, а на выходе выдавать батч вероятностей классов, прямо как в классификации картинок.

Каждый токен пройдёт вначале через эмбеддинг, а затем последовательность эмбеддингов пройдёт через LSTM. Самое последнее скрытое состояние будем считать векторным представлением для последовательности, поверх неё мы навесим линейный слой.

In [1]:
import torch
from torch import nn
import pandas as pd

class TextClassifier(nn.Module):
    def __init__(self, num_embeddings=25002, embedding_size=300, hidden_size=200, num_classes=2, num_layers=1):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
        

    def forward(self, x):
        embedded = self.embedding(x)
        _, (last_hidden, last_c) = self.lstm(embedded)  # doc on LSTM: cell and hidden state
        return self.linear(last_hidden[0]).squeeze()

model = TextClassifier()

In [2]:
model(torch.tensor([[1,2,3,4], [1,2,3,4]]))

tensor([0.0149, 0.0149], grad_fn=<SqueezeBackward0>)

## Скачаем и проинициализируем датасет

Дальше перейдём к торчтексту. Он почти как знакомый нам `torchvision` имеет внутри себя коллекцию датасетов для разных задач NLP в том числе и отзывы на IMDB. Инициализация датасета порой занимает время, не следует волноваться и судорожно перезапускать ядро.

In [3]:
import torchtext
from torchtext.legacy import data
from torchtext.legacy.datasets import IMDB


In [ ]:
# spacy -- вспомогательная библоитека для токенизации текста, скачаем токенайзер для английского языка
! pip install spacy  # transformers, natasha, torchtext
! python -m spacy download en_core_web_sm

В `torchtext` существует такая сущность, как `Field`. Это просто класс, в котором содержится описание колонок нашего датасета. В нашем случае всё довольно просто. Есть две колонки --- это текст и оценка, назовём их `text_field` и `label_field` соответственно. Токенизовать будем при помощи установленной только что библиотеки `spacy`, лейблы приведём к типу `float`. Разумеется, как и на прошлом семинаре ставим batch_first=False.

In [5]:

text_field = data.Field(tokenize='spacy',
                        batch_first=True,
                        include_lengths=False,
                        tokenizer_language='en_core_web_sm')

label_field = data.LabelField(dtype=torch.float32, batch_first=True)



Создадим два сплита нашего датасета. Они задаются при помощи метода splits.

In [8]:
data_train, data_test = IMDB.splits(text_field, label_field)

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:02<00:00, 35.1MB/s]


Создадим также словари, соответствующие нашему тексту. Выкинем все слова (токены), которые встречаются редко. Оставим только 25 тысяч самых частых слов.

Также согласуем номера токенов в словаре с эмбеддингами glove. При желании можно согласовать и с word2vec'ом, но имейте ввиду, что glove это их аналог. Об инициализации эмбеддингов мы обязательно поговорим чуть позже.

In [11]:
text_field.

In [12]:
vocab_size = 25000

# build_vocab -- создать словарь по данному полю в датасете
text_field.build_vocab(data_train,
                       max_size=vocab_size,
                       vectors="glove.6B.100d",
                      )

label_field.build_vocab(data_train)


for item in data_train:
    print(item.text)
    break

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:25<00:00, 15990.37it/s]


['I', 'recently', 'got', 'the', 'chance', 'to', 'view', '"', 'The', 'Waterdance', '"', ',', 'and', 'quite', 'liked', 'it', '.', 'I', 'do', "n't", 'really', 'understand', 'why', 'its', 'called', 'that', 'as', 'there', 'is', "n't", 'really', 'any', 'dancing', 'going', 'on', 'there', ',', 'except', 'maybe', 'for', 'the', 'dancing', 'at', 'the', 'strip', 'club', 'near', 'the', 'end', '.', 'We', 'are', 'introduced', 'to', 'the', 'main', 'characters', 'throughout', 'the', 'movie', ',', 'invalids', 'in', 'a', 'hospital', '.', 'The', 'story', 'shows', 'a', 'love', 'affair', 'between', 'a', 'physically', 'sisabled', 'guy', 'and', 'a', 'healthy', 'woman', ',', 'which', 'is', 'a', 'very', 'sweet', 'story', '.', 'Unfortunately', ',', 'you', 'do', "n't", 'get', 'to', 'see', 'movies', 'like', 'that', 'today', '.', 'I', 'm', 'not', '"', 'stuck', 'in', 'a', 'time', 'warp', '"', ',', 'i', 'm', 'not', 'saying', 'that', 'everything', 'during', 'the', '80s', 'and', 'early', '90s', 'was', 'better', 'than',

Давайте посмотрим на самые частые токены в словаре.

In [13]:
text_field.vocab.freqs.most_common(100)

[('the', 289838),
 (',', 275296),
 ('.', 236843),
 ('and', 156483),
 ('a', 156282),
 ('of', 144055),
 ('to', 133886),
 ('is', 109095),
 ('in', 87676),
 ('I', 77546),
 ('it', 76545),
 ('that', 70355),
 ('"', 63329),
 ("'s", 61928),
 ('this', 60483),
 ('-', 52863),
 ('/><br', 50935),
 ('was', 50013),
 ('as', 43508),
 ('with', 42807),
 ('movie', 42423),
 ('for', 42112),
 ('film', 38841),
 ('The', 37539),
 ('but', 35046),
 ('(', 33106),
 ('on', 32873),
 (')', 32850),
 ("n't", 32846),
 ('you', 30567),
 ('are', 29941),
 ('not', 28906),
 ('have', 28233),
 ('his', 27893),
 ('be', 26646),
 ('he', 24722),
 ('one', 24165),
 ('!', 21922),
 ('by', 21799),
 ('at', 21787),
 ('all', 21467),
 ('an', 20782),
 ('who', 20564),
 ('they', 19606),
 ('from', 19600),
 ('like', 19414),
 ('so', 17732),
 ('her', 17495),
 ("'", 17138),
 ('or', 17111),
 ('about', 17036),
 ('has', 17009),
 ('It', 16724),
 ('out', 16695),
 ('just', 16586),
 ('do', 15732),
 ('?', 14799),
 ('some', 14429),
 ('good', 14327),
 ('more', 1

In [14]:
# itos -- index to string
text_field.vocab.itos[:10]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']

torch.Size([25002, 100])

Как мы видим, в поле текста у нас стоят токены, на которые поделил spacy поделил текст! Осталось только воспользоваться построенным словарём и сделать аналог даталоадера по данным!

In [15]:
train_dataloader, test_dataloader = data.BucketIterator.splits((data_train, data_test), batch_size=32, device="cuda:0")

In [16]:
for item in train_dataloader:
    print(item.text.shape, item.label.shape)
    break

torch.Size([32, 1085]) torch.Size([32])


1077 это очень много для нашей маленькой модельки. Будем обрезать их на 256 токенах в нашем трейн лупе.

## Train loop

Перейдём к самому интересному (нет) и построим трейн луп к нашей модели. Но скажем сразу, он мало чем будет отличаться от классификации картинок.

In [19]:
def train_epoch(
    model,
    data_loader,
    optimizer,
    criterion,
    return_losses=False,
    device="cuda:0",
):
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])#.reshape((0, ))
    total_labels = np.array([])#.reshape((0, ))
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for item in data_loader:
            images = item.text
            labels = item.label

            # Move Batch to GPU
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            # Update weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            # Update descirption for tqdm
            accuracy = ((predicted > 0.5).int() == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions,(predicted > 0.5).int().cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    if return_losses:
        return metrics, all_losses
    else:
        return metrics


@torch.no_grad()
def validate(model, data_loader, criterion, device="cuda:0"):
    model = model.eval()
    total_loss = 0
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    return metrics

In [20]:
import numpy as np
from tqdm.notebook import tqdm
import sys

device="cuda:0"
model = TextClassifier()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

for i in range(3):
    train_epoch(model, train_dataloader, criterion=criterion, optimizer=optimizer, device=device)



  0%|          | 0/782 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

Кажется, обучается так себе. Давайте это исправлять! Попробуем сделать модель двунаправленной, а также увеличим число слоёв. Должно быть существенно лучше!

In [21]:
class TextClassifier(nn.Module):
    def __init__(self, num_embeddings=25002, embedding_size=300, hidden_size=200, num_classes=2, num_layers=1, pad_token=1):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_size, padding_idx=pad_token)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True, num_layers=num_layers, bidirectional=True)
        self.linear = nn.Linear(hidden_size * num_layers, 1)
        self.dropout = nn.Dropout(0.5)
        

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        _, (last_hidden, last_c) = self.lstm(embedded)
        hidden = torch.cat([last_hidden[-2], last_hidden[-1]], dim=1)
        return self.linear(hidden).squeeze()

device="cuda:0"
pad_token = text_field.vocab.stoi['<pad>']
model = TextClassifier(hidden_size=512, embedding_size=100, num_layers=2, pad_token=pad_token)
model.embedding.weight.data = text_field.vocab.vectors
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.BCEWithLogitsLoss()

for i in range(10):
    train_epoch(model, train_dataloader, criterion=criterion, optimizer=optimizer, device=device)

#model(torch.tensor([[1,2,3,4], [1,2,3,4]]))

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

In [22]:
validate(model, val_dataloader, criterion=criterion, optimizer=optimizer, device=device)

NameError: ignored

In [ ]:
ln(2)

In [ ]:
##### mlflow